# Creation of Publication Table of Results
 
 This notebook will reorganize data to create a final table of results to be submitted for publication. 
 For every globular cluster object there are multiple measurement files: one for each individual 
 chemical element that was measured, which contains the wavelength of the line 
 and the abundance measurement.  The final table will have rows corresponding to line 
 wavelengths and one column for each cluster that lists the measurement for that line.

 The final table will also require more information for each line wavelength, including
 its excitation potential and oscillator strength. This information is stored in a 
 separate file which will be read in and joined on each line.

 Note: not all clusters have measurements for a given element.  For clusters with measurements
 there is a different selection of line wavelengths measured, so the final table is
 expected to have many missing values.



In [1]:
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import glob
import pandas as pd
import string

In [3]:
# Array to loop over the elements of interest, filenames include element.
el_array=['ni','sc','v','cr','mn','co','cu','y','ba','la','nd','Eu']
#create array to hold information for all elements
biglist=[]

In [4]:
#loop over elements
for rr,pp in enumerate(el_array):
    datalist=[]
    #find all the files for given element
    sfiles=glob.glob("*"+pp+"*synth*")
    #loop over each file, one for each globular cluster
    for xx in sfiles:
        #pull out the cluster name to sort later
        cstr= xx.split("cmd")
        cname=cstr[0]
        #Read file:  wavelength of line, abundance solution, Flag if measurement
        # is an upper limit only
        abundtab=pd.read_table(xx,skipinitialspace=1,sep=' ',skiprows=1,
                               names=['line','solution','LIM'])
        #associate the measurements with the cluster in a new column
        abundtab['Name']=cname
        # combine
        datalist.append(abundtab)
        
    #concatenate list of measurement tables for all globular clusters
    el_abund=pd.concat(datalist)
    #Pivot on line to get the format we want
    format_el=el_abund.pivot(index='line',columns='Name',values='solution')
    #Make element capitalized, store the element in a column because the final table
    #will group wavelengths by element
    format_el['Element']=string.capwords(pp) 
    #combine for all elements
    biglist.append(format_el)

In [5]:
#sample of original file, which is the last one processed : Eu for g327
abundtab

,line,solution,LIM,Name
0,4129,-0.64,NaN,g327


In [6]:
#view of all cluster measurements for given element, last one processed: Eu, which has
#two possible wavelengths, 4129 and 6645
el_abund

,line,solution,LIM,Name
0,6645,0.06,NaN,b006
0,4129,-0.42,LIM,b012
0,6645,0.33,NaN,b029
0,6645,0.11,NaN,b045
0,6645,-0.17,LIM,b110
0,6645,-0.27,LIM,b171
0,4129,-0.61,NaN,b232
0,4129,-1.14,NaN,b358
0,6645,-0.12,NaN,b381
1,4129,-0.02,NaN,b381


In [7]:
#View large appended tables: one row for each wavelength. One column for each cluster
# one table for each element
biglist


[Name  b006  b012  b029  b034  b045  b048  b088  b110  b163  b171   ...     \
 line                                                               ...      
 5084   NaN   NaN  5.72   NaN   NaN  5.06   NaN   NaN   NaN   NaN   ...      
 5115   NaN   NaN  5.62   NaN   NaN  5.06   NaN   NaN   NaN   NaN   ...      
 5435   NaN   NaN   NaN   NaN  5.61  5.26   NaN   NaN   NaN   NaN   ...      
 5476   NaN  4.68   NaN   NaN   NaN   NaN  4.58   NaN   NaN   NaN   ...      
 5578  5.51   NaN  5.82  5.06   NaN  5.26   NaN  5.35  5.75  5.66   ...      
 5593  5.51   NaN  5.92  5.36  5.51   NaN   NaN   NaN   NaN   NaN   ...      
 6086  5.51   NaN  5.92   NaN   NaN   NaN   NaN   NaN  5.75  5.66   ...      
 6108  5.61   NaN   NaN   NaN   NaN  5.26   NaN   NaN   NaN   NaN   ...      
 6128  5.51   NaN  5.82  5.56  5.01   NaN   NaN   NaN   NaN   NaN   ...      
 6327  5.61   NaN  5.72  5.36  5.21  5.36   NaN  5.65  5.75  5.76   ...      
 6339  5.41   NaN   NaN  5.36  4.91   NaN   NaN   NaN   NaN   Na

In [8]:
# concatenate list of tables into one table
measurements=pd.concat(biglist)
measurements


,Element,b006,b012,b029,b034,b045,b048,b088,b110,b163,...,b384,b386,b403,b405,b457,b514,g002,g327,h10,mgc1
line,,,,,,,,,,,,,,,,,,,,,
5084.0,Ni,NaN,NaN,5.72,NaN,NaN,5.06,NaN,NaN,NaN,...,5.24,4.98,NaN,NaN,NaN,4.22,NaN,NaN,4.58,4.43
5115.0,Ni,NaN,NaN,5.62,NaN,NaN,5.06,NaN,NaN,NaN,...,5.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.48,NaN
5435.0,Ni,NaN,NaN,NaN,NaN,5.61,5.26,NaN,NaN,NaN,...,5.74,5.38,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5476.0,Ni,NaN,4.68,NaN,NaN,NaN,NaN,4.58,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.62,4.54,4.73,NaN,NaN
5578.0,Ni,5.51,NaN,5.82,5.06,NaN,5.26,NaN,5.35,5.75,...,5.34,4.88,5.30,NaN,NaN,4.42,NaN,NaN,4.58,NaN
5593.0,Ni,5.51,NaN,5.92,5.36,5.51,NaN,NaN,NaN,NaN,...,5.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6086.0,Ni,5.51,NaN,5.92,NaN,NaN,NaN,NaN,NaN,5.75,...,5.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6108.0,Ni,5.61,NaN,NaN,NaN,NaN,5.26,NaN,NaN,NaN,...,5.34,NaN,5.20,5.06,4.83,NaN,NaN,4.43,4.48,NaN
6128.0,Ni,5.51,NaN,5.82,5.56,5.01,NaN,NaN,NaN,NaN,...,5.34,NaN,5.70,NaN,NaN,NaN,NaN,NaN,4.68,NaN


In [9]:
#Change wavelength from index to regular column
reindex_measurement=measurements.reset_index(level=0)
reindex_measurement

,line,Element,b006,b012,b029,b034,b045,b048,b088,b110,...,b384,b386,b403,b405,b457,b514,g002,g327,h10,mgc1
0,5084.0,Ni,NaN,NaN,5.72,NaN,NaN,5.06,NaN,NaN,...,5.24,4.98,NaN,NaN,NaN,4.22,NaN,NaN,4.58,4.43
1,5115.0,Ni,NaN,NaN,5.62,NaN,NaN,5.06,NaN,NaN,...,5.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.48,NaN
2,5435.0,Ni,NaN,NaN,NaN,NaN,5.61,5.26,NaN,NaN,...,5.74,5.38,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5476.0,Ni,NaN,4.68,NaN,NaN,NaN,NaN,4.58,NaN,...,NaN,NaN,NaN,NaN,NaN,4.62,4.54,4.73,NaN,NaN
4,5578.0,Ni,5.51,NaN,5.82,5.06,NaN,5.26,NaN,5.35,...,5.34,4.88,5.30,NaN,NaN,4.42,NaN,NaN,4.58,NaN
5,5593.0,Ni,5.51,NaN,5.92,5.36,5.51,NaN,NaN,NaN,...,5.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6086.0,Ni,5.51,NaN,5.92,NaN,NaN,NaN,NaN,NaN,...,5.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6108.0,Ni,5.61,NaN,NaN,NaN,NaN,5.26,NaN,NaN,...,5.34,NaN,5.20,5.06,4.83,NaN,NaN,4.43,4.48,NaN
8,6128.0,Ni,5.51,NaN,5.82,5.56,5.01,NaN,NaN,NaN,...,5.34,NaN,5.70,NaN,NaN,NaN,NaN,NaN,4.68,NaN
9,6327.0,Ni,5.61,NaN,5.72,5.36,5.21,5.36,NaN,5.65,...,5.64,5.18,5.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# read in the file with the line atomic information. Rename the Element column so it can
# be joined to the other dataframe.
line_info=pd.read_table('atomic_information.txt',skipinitialspace=1,sep=' ',skip_blank_lines=True)
line_info = line_info.rename(columns={ 'Ele' : 'Element'})
line_info

,Element,Ion,wave,EP,logGF
0,Na,I,5682.650,2.100,-0.700
1,Na,I,5688.220,2.100,-0.460
2,Na,I,6154.230,2.100,-1.570
3,Na,I,6160.750,2.100,-1.270
4,Mg,I,4167.280,4.350,-1.000
5,Mg,I,4351.920,4.350,-0.520
6,Mg,I,4571.100,0.000,-5.570
7,Mg,I,4703.000,4.350,-0.380
8,Mg,I,5183.620,2.720,-0.160
9,Mg,I,5528.420,4.350,-0.340


In [11]:
# The measurement tables only have wavelengths to whole Angstroms. We ultimately want
# the wavelength to hundredths of Angstroms as listed in line_info, but we will
# join the tables on whole angstroms.
# 
# Therefore need a new dummy column with whole angstrom wavelengths in line_info for join
# We also have to change the type  from string to int64 to match other table.
line_info.wave.astype(str).str[0:4]
line_info['sline']=line_info.wave.astype(str).str[0:4]
line_info['line']=line_info['sline'].astype('int64')
line_info['line'].dtype
line_info=line_info.drop('sline',1)
#line_info

In [12]:
# Now merge two tables on the wavelength and Element. Use Left join because we don't need
# information for lines that had no measurements. Do not sort to keep lines
# grouped by Element
bigtable=pd.merge(reindex_measurement,line_info,on=('line','Element'),
                  how='left',sort=False)
bigtable

,line,Element,b006,b012,b029,b034,b045,b048,b088,b110,...,b457,b514,g002,g327,h10,mgc1,Ion,wave,EP,logGF
0,5084.0,Ni,NaN,NaN,5.72,NaN,NaN,5.06,NaN,NaN,...,NaN,4.22,NaN,NaN,4.58,4.43,I,5084.090,3.680,0.03
1,5115.0,Ni,NaN,NaN,5.62,NaN,NaN,5.06,NaN,NaN,...,NaN,NaN,NaN,NaN,4.48,NaN,I,5115.390,3.830,-0.11
2,5435.0,Ni,NaN,NaN,NaN,NaN,5.61,5.26,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,I,5435.880,1.990,-2.59
3,5476.0,Ni,NaN,4.68,NaN,NaN,NaN,NaN,4.58,NaN,...,NaN,4.62,4.54,4.73,NaN,NaN,I,5476.920,1.830,-0.89
4,5578.0,Ni,5.51,NaN,5.82,5.06,NaN,5.26,NaN,5.35,...,NaN,4.42,NaN,NaN,4.58,NaN,I,5578.730,1.680,-2.67
5,5593.0,Ni,5.51,NaN,5.92,5.36,5.51,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,I,5593.750,3.900,-0.79
6,6086.0,Ni,5.51,NaN,5.92,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,I,6086.290,4.270,-0.47
7,6108.0,Ni,5.61,NaN,NaN,NaN,NaN,5.26,NaN,NaN,...,4.83,NaN,NaN,4.43,4.48,NaN,I,6108.130,1.680,-2.45
8,6128.0,Ni,5.51,NaN,5.82,5.56,5.01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.68,NaN,I,6128.980,1.680,-3.39
9,6327.0,Ni,5.61,NaN,5.72,5.36,5.21,5.36,NaN,5.65,...,NaN,NaN,NaN,NaN,NaN,NaN,I,6327.604,1.676,-3.15


In [13]:
#cols=list(bigtable.columns.values)

In [14]:
# reorder columns the way we want them at publication, drop dummy column
sorted_table=bigtable[['Element','Ion','wave','EP','logGF','b006',
 'b012',
 'b029',
 'b034',
 'b045',
 'b048',
 'b088',
 'b110',
 'b163',
 'b171',
 'b182',
 'b193',
 'b225',
 'b232',
 'b235',
 'b240',
 'b311',
 'b312',
 'b358',
 'b381',
 'b383',
 'b384',
 'b386',
 'b403',
 'b405',
 'b457',
 'b514',
 'g002',
 'g327',
 'h10',
 'mgc1',]]

In [15]:
#Here change the data types of wavelength, EP and logGF to numeric so we can format them.
sorted_table=sorted_table.apply(lambda x: pd.to_numeric(x, errors='ignore'))

In [16]:
# Format columns
sorted_table['wave']=sorted_table['wave'].map('{:.2f}'.format)
sorted_table['EP']=sorted_table['EP'].map('{:.2f}'.format)
sorted_table['logGF']=sorted_table['logGF'].map('{:.2f}'.format)
sorted_table

,Element,Ion,wave,EP,logGF,b006,b012,b029,b034,b045,...,b384,b386,b403,b405,b457,b514,g002,g327,h10,mgc1
0,Ni,I,5084.09,3.68,0.03,NaN,NaN,5.72,NaN,NaN,...,5.24,4.98,NaN,NaN,NaN,4.22,NaN,NaN,4.58,4.43
1,Ni,I,5115.39,3.83,-0.11,NaN,NaN,5.62,NaN,NaN,...,5.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.48,NaN
2,Ni,I,5435.88,1.99,-2.59,NaN,NaN,NaN,NaN,5.61,...,5.74,5.38,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ni,I,5476.92,1.83,-0.89,NaN,4.68,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.62,4.54,4.73,NaN,NaN
4,Ni,I,5578.73,1.68,-2.67,5.51,NaN,5.82,5.06,NaN,...,5.34,4.88,5.30,NaN,NaN,4.42,NaN,NaN,4.58,NaN
5,Ni,I,5593.75,3.90,-0.79,5.51,NaN,5.92,5.36,5.51,...,5.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ni,I,6086.29,4.27,-0.47,5.51,NaN,5.92,NaN,NaN,...,5.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Ni,I,6108.13,1.68,-2.45,5.61,NaN,NaN,NaN,NaN,...,5.34,NaN,5.20,5.06,4.83,NaN,NaN,4.43,4.48,NaN
8,Ni,I,6128.98,1.68,-3.39,5.51,NaN,5.82,5.56,5.01,...,5.34,NaN,5.70,NaN,NaN,NaN,NaN,NaN,4.68,NaN
9,Ni,I,6327.60,1.68,-3.15,5.61,NaN,5.72,5.36,5.21,...,5.64,5.18,5.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# save the dataframe as a latex table, fill NaN with -999.0
(sorted_table.to_latex('element_measurements.tex',na_rep='-999.0',index=False))


In [ ]:
#sorted_table.to_csv('pd_nf_table.txt',sep=' ', index=False)
